<a href="https://colab.research.google.com/github/juanserrano90/codelatam/blob/main/DataPreparation/spike_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>  Analysis on Telluric lines  </h1>

<center> Created on November 18th, 2024

Last modified on November 18th, 2024

Author: Paula GM </center>


---

The purpose of this notebook is to perform a statistical analysis of the Telluric lines present in the provided sn_parquet dataset and determine what should be categorized as a spike.

In [ ]:
import sys
from os.path import isfile

import numpy as np
import pandas as pd
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

# from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
url = "https://github.com/juanserrano90/codelatam/raw/main/data/sn_data.parquet"
df_raw = pd.read_parquet(url)

In [ ]:
df_raw.head()

In [ ]:
wavelength = np.array([float(c) for c in df_raw.columns[5:]])